# 2020_dl_anomaly_detector

This is an experiment with deep learning for anomaly detection.

In [27]:
import torch

### Define the Data with Labels

In [28]:
train_data = torch.randn(128, 256)
valid_data = torch.randn(32, 256)
test_data = torch.randn(32, 256)

train_labels = torch.zeros(128)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

### Visualize the Data

In [33]:
dataiter = iter(train_loader)
print(dataiter.next().shape, dataiter.next())

torch.Size([32, 256]) tensor([[-0.1218, -0.2056,  0.6488,  ...,  0.0632, -0.2153, -3.0068],
        [ 0.6553, -0.6741, -1.2872,  ..., -0.7176,  0.4842, -0.0557],
        [-2.0957, -0.5742,  0.7494,  ...,  0.1331, -0.5559, -0.2624],
        ...,
        [ 0.4962, -0.0433,  1.1756,  ..., -0.3568, -0.3087, -1.1618],
        [ 0.5801, -0.0560, -0.1779,  ..., -0.7236,  2.5551,  1.2029],
        [-1.5716,  0.2455,  0.4063,  ..., -1.0507, -1.0109, -0.5023]])


### Define the Model

In [37]:
from torch import nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        
        return F.softmax(self.fc2(x), dim=1)
    
model = Model()
model

Model(
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [38]:
error_funct = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

### Train the Model

In [40]:
epochs = 1

for e in range(epochs):
    
    for data in train_loader:
        
        print(data.shape)
        probs = model(data)

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
